<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/unified_ai_secure_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# unified_ai_secure.py
# Runs: NLP QA, Image Captioning, Symbolic Reasoning, RL (Q-learning) -> collects results
# Packages artifacts -> encrypts with passphrase. Also supports decryption.

import argparse
import base64
import io
import json
import os
import tarfile
from dataclasses import dataclass, asdict
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np

# ---------- Optional deps ----------
try:
    from transformers import pipeline as hf_pipeline  # type: ignore
    HF_OK = True
except Exception as e:
    HF_OK = False

try:
    from PIL import Image  # type: ignore
    PIL_OK = True
except Exception:
    PIL_OK = False

try:
    import requests  # type: ignore
    REQUESTS_OK = True
except Exception:
    REQUESTS_OK = False

# RL env: prefer gymnasium; fallback to gym
GYM_API = None
try:
    import gymnasium as gym  # type: ignore
    GYM_API = "gymnasium"
except Exception:
    try:
        import gym  # type: ignore
        GYM_API = "gym"
    except Exception:
        GYM_API = None

# Crypto
try:
    from cryptography.hazmat.primitives import hashes  # type: ignore
    from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC  # type: ignore
    from cryptography.fernet import Fernet  # type: ignore
    CRYPTO_OK = True
except Exception:
    CRYPTO_OK = False

# ---------- Data structures ----------
@dataclass
class NLPResult:
    question: str
    answer: Optional[str]
    error: Optional[str]

@dataclass
class VisionResult:
    source: str
    caption: Optional[str]
    error: Optional[str]

@dataclass
class ReasoningResult:
    premises: List[str]
    query: str
    entailed: bool

@dataclass
class RLLog:
    env: str
    steps: int
    total_reward: float
    episode_terminated: bool
    episode_truncated: bool
    notes: str

@dataclass
class RunSummary:
    timestamp_utc: str
    seed: int
    nlp: NLPResult
    vision: VisionResult
    reasoning: ReasoningResult
    rl: RLLog
    notes: List[str]

# ---------- Helpers ----------
def ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def utcnow() -> str:
    return datetime.utcnow().isoformat() + "Z"

# ---------- NLP: QA ----------
def run_qa(context: str, question: str) -> NLPResult:
    if not HF_OK:
        return NLPResult(question, None, "transformers not available")
    try:
        qa = hf_pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
        out = qa(question=question, context=context)
        return NLPResult(question, out.get("answer"), None)
    except Exception as e:
        return NLPResult(question, None, f"{type(e).__name__}: {e}")

# ---------- Vision: BLIP caption ----------
def load_image_from_source(source: str) -> Image.Image:
    if source.startswith("http://") or source.startswith("https://"):
        if not REQUESTS_OK:
            raise RuntimeError("requests not available to fetch URL")
        resp = requests.get(source, timeout=30)
        resp.raise_for_status()
        from io import BytesIO
        return Image.open(BytesIO(resp.content)).convert("RGB")
    # Local path
    if not PIL_OK:
        raise RuntimeError("Pillow not available to open local image")
    return Image.open(source).convert("RGB")

def run_caption(image_source: Optional[str]) -> VisionResult:
    if image_source is None:
        return VisionResult(source="(none)", caption=None, error="no image source provided")
    if not HF_OK:
        return VisionResult(image_source, None, "transformers not available")
    try:
        image = load_image_from_source(image_source)
        cap = hf_pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
        res = cap(image)
        caption = res[0].get("generated_text") if res else None
        return VisionResult(image_source, caption, None)
    except Exception as e:
        return VisionResult(image_source, None, f"{type(e).__name__}: {e}")

# ---------- Symbolic reasoning (very simple syllogism) ----------
# Supports forms like: "all humans are mortal", "Socrates is human" -> query "Socrates is mortal"
def normalize_text(s: str) -> str:
    return " ".join(s.strip().lower().split())

def parse_all_are(p: str) -> Optional[Tuple[str, str]]:
    # "all X are Y" -> (X, Y)
    p = normalize_text(p)
    if not p.startswith("all ") or " are " not in p:
        return None
    core = p[4:]
    parts = core.split(" are ")
    if len(parts) != 2:
        return None
    return parts[0].strip(), parts[1].strip()

def parse_is(p: str) -> Optional[Tuple[str, str]]:
    # "a is b" -> (a, b)
    p = normalize_text(p)
    if " is " not in p:
        return None
    parts = p.split(" is ")
    if len(parts) != 2:
        return None
    return parts[0].strip(), parts[1].strip()

def run_reasoning(premises: List[str], query: str) -> ReasoningResult:
    alls: List[Tuple[str, str]] = []
    facts: List[Tuple[str, str]] = []
    for pr in premises:
        aa = parse_all_are(pr)
        if aa:
            alls.append(aa)
            continue
        ii = parse_is(pr)
        if ii:
            facts.append(ii)
    q = parse_is(query)
    entailed = False
    if q:
        a, b = q
        # If a is X and all X are b -> a is b
        xs = [x for (subj, x) in facts if subj == a]
        implied: List[str] = []
        for x in xs:
            implied += [B for (X, B) in alls if X == x]
        entailed = b in implied or (a, b) in facts
    return ReasoningResult(premises=premises, query=query, entailed=entailed)

# ---------- RL: Q-learning ----------
class QLearningAgent:
    def __init__(self, actions: List[int], lr=0.1, gamma=0.99, epsilon=0.1):
        self.actions = actions
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.q: Dict[str, Dict[int, float]] = {}

    def _ensure_state(self, s: Any):
        key = str(s)
        if key not in self.q:
            self.q[key] = {a: 0.0 for a in self.actions}
        return key

    def act(self, state: Any, rng: np.random.Generator) -> int:
        if rng.random() < self.epsilon:
            return int(rng.choice(self.actions))
        key = self._ensure_state(state)
        return max(self.q[key], key=self.q[key].get)

    def update(self, s: Any, a: int, r: float, s2: Any):
        ks = self._ensure_state(s)
        ks2 = self._ensure_state(s2)
        qsa = self.q[ks][a]
        max_next = max(self.q[ks2].values())
        self.q[ks][a] = qsa + self.lr * (r + self.gamma * max_next - qsa)

def make_env(seed: int = 7):
    if GYM_API == "gymnasium":
        env = gym.make("FrozenLake-v1", is_slippery=True)
        obs, info = env.reset(seed=seed)
        return env, "gymnasium", obs
    elif GYM_API == "gym":
        env = gym.make("FrozenLake-v1", is_slippery=True)
        try:
            obs = env.reset(seed=seed)
        except TypeError:
            obs = env.reset()
        return env, "gym", obs
    else:
        return None, None, None

def step_env(env, api: str, action: int):
    if api == "gymnasium":
        obs, reward, terminated, truncated, info = env.step(action)
        done = bool(terminated or truncated)
        return obs, float(reward), done, bool(terminated), bool(truncated)
    else:
        obs, reward, done, info = env.step(action)
        return obs, float(reward), bool(done), bool(done), False

def run_rl_episode(max_steps=50, seed=7) -> RLLog:
    env, api, obs = make_env(seed=seed)
    if env is None:
        return RLLog(env="(unavailable)", steps=0, total_reward=0.0,
                     episode_terminated=False, episode_truncated=False,
                     notes="gym/gymnasium not available")
    rng = np.random.default_rng(seed)
    agent = QLearningAgent(actions=[0, 1, 2, 3], lr=0.2, gamma=0.95, epsilon=0.2)
    total_reward = 0.0
    steps = 0
    terminated = False
    truncated = False
    state = obs
    for t in range(max_steps):
        act = agent.act(state, rng)
        next_state, reward, done, term, trunc = step_env(env, api, act)
        agent.update(state, act, reward, next_state)
        total_reward += reward
        steps = t + 1
        state = next_state
        if done:
            terminated, truncated = term, trunc
            break
    try:
        env.close()
    except Exception:
        pass
    return RLLog(env=f"FrozenLake-v1 ({api})", steps=steps, total_reward=total_reward,
                 episode_terminated=terminated, episode_truncated=truncated,
                 notes="Single-episode demo with tabular Q-learning")

# ---------- Encryption packaging ----------
def derive_key(passphrase: str, salt: bytes) -> bytes:
    kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), length=32, salt=salt, iterations=390000)
    key = kdf.derive(passphrase.encode("utf-8"))
    return base64.urlsafe_b64encode(key)

def encrypt_bytes(payload: bytes, passphrase: str) -> Dict[str, str]:
    if not CRYPTO_OK:
        raise RuntimeError("cryptography not installed. pip install cryptography")
    import secrets
    salt = secrets.token_bytes(16)
    fkey = derive_key(passphrase, salt)
    token = Fernet(fkey).encrypt(payload)
    return {
        "format": "unified.ai.enc.v1",
        "salt_b64": base64.urlsafe_b64encode(salt).decode("ascii"),
        "token_b64": base64.urlsafe_b64encode(token).decode("ascii"),
        "created_utc": utcnow()
    }

def decrypt_bytes(obj: Dict[str, str], passphrase: str) -> bytes:
    if not CRYPTO_OK:
        raise RuntimeError("cryptography not installed. pip install cryptography")
    if obj.get("format") != "unified.ai.enc.v1":
        raise ValueError("Unsupported format")
    salt = base64.urlsafe_b64decode(obj["salt_b64"].encode("ascii"))
    token = base64.urlsafe_b64decode(obj["token_b64"].encode("ascii"))
    key = derive_key(passphrase, salt)
    return Fernet(key).decrypt(token)

def tar_dir_to_bytes(root: Path) -> bytes:
    buf = io.BytesIO()
    with tarfile.open(fileobj=buf, mode="w:gz") as tf:
        for dirpath, _, filenames in os.walk(root):
            for fname in filenames:
                full = Path(dirpath) / fname
                arc = full.relative_to(root)
                tf.add(full, arcname=str(arc))
    buf.seek(0)
    return buf.read()

def extract_tar_bytes(payload: bytes, outdir: Path) -> None:
    with tarfile.open(fileobj=io.BytesIO(payload), mode="r:gz") as tf:
        tf.extractall(path=outdir)

# ---------- Pipeline ----------
def run_pipeline(workdir: Path,
                 image_source: Optional[str],
                 seed: int) -> Tuple[Path, RunSummary]:
    ensure_dir(workdir)
    artifacts = workdir / f"artifacts_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}"
    ensure_dir(artifacts)

    notes: List[str] = []

    # NLP
    context = "France is a country in Western Europe. Its capital, Paris, is known for art and culture."
    question = "What is the capital of France?"
    nlp_res = run_qa(context, question)
    if nlp_res.error:
        notes.append(f"NLP skipped/error: {nlp_res.error}")

    # Vision
    vis_res = run_caption(image_source)
    if vis_res.error:
        notes.append(f"Vision skipped/error: {vis_res.error}")

    # Reasoning
    premises = ["all humans are mortal", "Socrates is human"]
    query = "Socrates is mortal"
    reason_res = run_reasoning(premises, query)

    # RL
    rl_res = run_rl_episode(max_steps=50, seed=seed)
    if "unavailable" in rl_res.env:
        notes.append("RL skipped: gym/gymnasium not available")

    summary = RunSummary(
        timestamp_utc=utcnow(),
        seed=seed,
        nlp=nlp_res,
        vision=vis_res,
        reasoning=reason_res,
        rl=rl_res,
        notes=notes
    )

    # Save JSON result
    (artifacts / "results.json").write_text(json.dumps(asdict(summary), indent=2))
    # Lightweight README
    (artifacts / "README.txt").write_text(
        "Unified AI run results. Contains results.json with outputs for NLP, vision, reasoning, and RL.\n"
    )
    return artifacts, summary

def encrypt_artifacts(artifacts_dir: Path, passphrase: str, out_path: Path) -> Path:
    payload = tar_dir_to_bytes(artifacts_dir)
    enc = encrypt_bytes(payload, passphrase)
    out_path.write_text(json.dumps(enc, indent=2))
    # Write a helper README in the artifacts dir
    (artifacts_dir / "ENCRYPTION_README.txt").write_text(
        f"Encrypted package created at: {out_path}\n"
        f"Decrypt with:\n"
        f"  python unified_ai_secure.py --decrypt --encrypted \"{out_path}\" "
        f"--passphrase \"YOUR-PASSPHRASE\" --outdir \"./decrypted\"\n"
    )
    return out_path

def decrypt_package(enc_path: Path, passphrase: str, outdir: Path) -> None:
    obj = json.loads(enc_path.read_text())
    payload = decrypt_bytes(obj, passphrase)
    outdir.mkdir(parents=True, exist_ok=True)
    extract_tar_bytes(payload, outdir)
    (outdir / "DECRYPT_OK.txt").write_text("Decryption successful.\n")

# ---------- CLI ----------
def main():
    ap = argparse.ArgumentParser(description="Unified AI system with encryption (NLP, Vision, Reasoning, RL)")
    ap.add_argument("--workdir", type=Path, default=Path("/mnt/data/unified_ai"))
    ap.add_argument("--run-all", action="store_true", help="Run all modules and package artifacts")
    ap.add_argument("--image", type=str, default=None, help="Image URL or local path for captioning")
    ap.add_argument("--seed", type=int, default=7, help="Seed for RL and stochastic choices")
    ap.add_argument("--passphrase", type=str, default=None, help="Passphrase for encryption/decryption")
    ap.add_argument("--decrypt", action="store_true", help="Decrypt an encrypted package")
    ap.add_argument("--encrypted", type=Path, help="Path to encrypted .enc.json")
    ap.add_argument("--outdir", type=Path, help="Directory to extract decrypted contents")
    args = ap.parse_args()

    if args.decrypt:
        if not args.passphrase:
            raise SystemExit("ERROR: --passphrase is required for --decrypt")
        if not args.encrypted:
            raise SystemExit("ERROR: --encrypted path is required for --decrypt")
        outdir = args.outdir or (args.workdir / "decrypted")
        decrypt_package(args.encrypted, args.passphrase, outdir)
        print(f"[OK] Decrypted to: {outdir}")
        return

    if args.run_all:
        artifacts, summary = run_pipeline(args.workdir, args.image, args.seed)
        if args.passphrase:
            enc_out = args.workdir / "ua_artifacts.enc.json"
            encrypt_artifacts(artifacts, args.passphrase, enc_out)
            print(f"[OK] Encrypted package: {enc_out}")
        else:
            print("[NOTE] No passphrase provided; artifacts left unencrypted.")
        print(f"[OK] Artifacts directory: {artifacts}")
        # Friendly echo of key results
        print("--- Summary ---")
        print(f"NLP: {summary.nlp.answer or summary.nlp.error}")
        print(f"Vision: {summary.vision.caption or summary.vision.error}")
        print(f"Reasoning: {summary.reasoning.query} -> {summary.reasoning.entailed}")
        print(f"RL: reward={summary.rl.total_reward:.2f}, steps={summary.rl.steps}, env={summary.rl.env}")
        return

    ap.print_help()

if __name__ == "__main__":
    main()